# 待解問題
- [X] 自動下拉距離
- [X] 內文格式不同的狀況
- [X] 浮文問題
- [ ] 發文日期
- [X] 寫進excel

In [1]:
import re
import pandas as pd
import time
from datetime import datetime
from collections import defaultdict
from selenium import webdriver
from selenium.common.exceptions import ElementNotVisibleException

In [2]:
def last_timestamp(timestamp_list):
    
    timestamp = []
    
    for ts in timestamp_list:
        if ts.text != '':
            timestamp.append(ts.text)

    last_timestamp = timestamp[-1]
    
    return last_timestamp

In [3]:
def date_of_post():
    
    date_class = ".b6zbclly.myohyog2.l9j0dhe7.aenfhxwr.l94mrbxd.ihxqhq3m.nc684nl6.t5a262vz.sdhka5h4"
    last_height = browser.execute_script("return document.body.scrollHeight")
    timestamp_list = browser.find_elements_by_css_selector(date_class)

    last_ts = last_timestamp(timestamp_list)
    print(last_ts)
    
    if re.search(r'小時|2020年12月19日', last_ts):
        return True 
    else:
        new_height = last_height + 500
        browser.execute_script(f"window.scrollTo(0, {new_height});")
        timestamp_list_check = browser.find_elements_by_css_selector(date_class)
        last_ts = last_timestamp(timestamp_list_check)
        print(last_ts)
        
        if re.search(r'小時|2020年12月19日', last_ts):
            return True
        else:    
            return False

In [4]:
def see_more():

    click_string = "div[class='oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl oo9gr5id gpro0wi8 lrazzd5p'][role='button']"

    try:
        read_button = browser.find_element_by_css_selector(click_string)
        time.sleep(0.5)
        if read_button:
            read_button.click()
    except ElementNotVisibleException:
        print("Exception：element not visible")
    except Exception as e:
        print(Exception, e)

In [21]:
# 內容
def get_company_content(company, browser):
    
    titles = browser.find_elements_by_css_selector("div[class='kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q']")
    values = browser.find_elements_by_css_selector("div[class='o9v6fnle cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q']")
    #timestamps = browser.find_elements_by_css_selector(".b6zbclly.myohyog2.l9j0dhe7.aenfhxwr.l94mrbxd.ihxqhq3m.nc684nl6.t5a262vz.sdhka5h4")
    
    # title
    for title in titles:
        if title.text != '':
            content = title.text
            
            if re.search(r'的真實價值', content):
                sub_content = re.split(',|、|的真實價值',content)
                for i, sub in enumerate(sub_content):
                    #print(i, sub)
                    code = re.search(r'\d{4}', sub)                        
                    if code:
                        code = code.group()
                        name = re.split(r'\(|（', sub)[0].strip()
                        name = re.sub(r'.\n', '', name)
                        company[name]['Code'] = code
                        
    # value            
    for value in values:
        if value.text != '':
            infos = value.text.split('\n')

            for i, info in enumerate(infos):
                #print(i, info)

                if re.search(r'的真實價值', info):
                    code = re.search(r'\d{4}', info)
                    if code:
                        code = code.group()
                        name = re.split(r'\(|（', info)[0].strip()
                        name = re.sub(r'.\n', '', name)
                        company[name]['Code'] = code

                elif re.search(r'的 Ks =', info):
                    index = re.split('的 Ks =', info)
                    index_name = index[0].strip()
                    index_value = index[1].strip()
                    company[index_name]['Ks'] = index_value

                elif re.search(r'的 P / E =', info):
                    index = re.split('的 P / E =', info)
                    index_name = index[0].strip()
                    index_value = index[1].split('=')[-1].strip()
                    company[index_name]['P / E'] = index_value

                elif re.search(r'的 FRV =', info):
                    index = re.split('的 FRV =', info)
                    index_name = index[0].strip()
                    index_value = index[1].split('=')[-1].strip()
                    company[index_name]['FRV'] = index_value

                elif re.search(r'的 泡沫程度 =', info):
                    index = re.split('的 泡沫程度 =', info)
                    index_name = index[0].strip()
                    index_value = index[1].split('=')[-1].strip()
                    company[index_name]['泡沫程度'] = index_value
            
    return company

In [22]:
def main():

    post = date_of_post()
    last_height = browser.execute_script("return document.body.scrollHeight")
    time.sleep(2)

    while(post):
        
        new_height = last_height + 500
        browser.execute_script(f"window.scrollTo(0, {new_height});")
        time.sleep(2)

        # 確認目前下拉範圍
        post = date_of_post()

    if not post:
        
        last_height = browser.execute_script("return document.body.scrollHeight")
        browser.execute_script(f"window.scrollTo(0, {last_height});")
        time.sleep(1)
        
        browser.execute_script("window.scrollTo(0, 0);")

        check_height = 0
        while(check_height < last_height):
            browser.execute_script(f"window.scrollTo(0, {check_height});")
            time.sleep(2)
            see_more()
            
            get_company_content(company, browser)
            check_height += 500

In [23]:
browser = webdriver.Chrome()
browser.get('https://www.facebook.com/groups/1189431674567215/')
company = defaultdict(dict)

In [24]:
%%time
main()

2020年12月19日
6月12日下午11:00
6月12日下午11:00
Exception：element not visible
Exception：element not visible
Exception：element not visible
Exception：element not visible
Exception：element not visible
Wall time: 1min 10s


In [25]:
company

defaultdict(dict,
            {'奇偶': {'Code': '3356',
              'Ks': '0.052',
              'P / E': '19.23',
              'FRV': '50.57',
              '泡沫程度': '0.577'},
             '志聖': {'Code': '2467',
              'Ks': '0.047',
              'P / E': '21.28',
              'FRV': '80.44',
              '泡沫程度': '0.656'},
             '群翊': {'Code': '6664',
              'Ks': '0.056',
              'P / E': '17.86',
              'FRV': '124.31',
              '泡沫程度': '0.559'},
             '迅得': {'Code': '6438',
              'Ks': '0.052',
              'P / E': '19.23',
              'FRV': '147.69',
              '泡沫程度': '0.662'},
             '大量': {'Code': '3167',
              'Ks': '0.056',
              'P / E': '17.86',
              'FRV': '66.62',
              '泡沫程度': '0.937'},
             '大田': {'Code': '8924',
              'Ks': '0.050',
              'P / E': '20.00',
              'FRV': '265.60',
              '泡沫程度': '0.616'},
             '申豐': {'Code

In [26]:
def to_file(company):
    
    df_list = []
    for com in company.keys():
        if len(company[com].keys()) == 5:
            company[com]['Name'] = com
            df_list.append(company[com])

    df = pd.DataFrame(df_list, columns=['Name', 'Code', 'Ks', 'P / E', 'FRV', '泡沫程度'])
    current_date = datetime.now().strftime('%Y%m%d')
    df.to_csv(f'./{current_date}.csv', index=False, encoding="utf-8-sig")

In [27]:
to_file(company)